In [11]:
!uv pip install google-genai python-dotenv pytz

Audited 3 packages in 10ms


In [ ]:
import os

from dotenv import load_dotenv
from google import genai
from google.genai import types

load_dotenv()

True

In [ ]:
client = genai.Client(api_key=os.getenv("FREE_STUDIO_KEY"))

with open("./system_prompt_2.md", "r", encoding="utf-8") as f:
    system_prompt = f.read()

In [13]:
MODEL = "gemini-2.5-flash-preview-09-2025"

In [14]:
response_schema = types.Schema(
    type=types.Type.OBJECT,
    properties={
        "chain_of_thought": types.Schema(type=types.Type.STRING),
        "estimated_mw": types.Schema(type=types.Type.INTEGER),
        "score": types.Schema(type=types.Type.NUMBER, maximum=10.0, minimum=0.0),
        "confidence": types.Schema(type=types.Type.INTEGER, maximum=100, minimum=0),
    },
    required=["chain_of_thought", "estimated_mw", "score", "confidence"],
)

In [23]:
# cached_content = client.caches.create(
#     model=MODEL,
#     config=types.CreateCachedContentConfig(
#         system_instruction=system_prompt,
#         expire_time=datetime(
#             year=2025, month=12, day=5, tzinfo=pytz.timezone("Asia/Kolkata")
#         ),
#         display_name="score-cache",
#     ),
# )

In [ ]:
# Example usage
prompts = [
    "Analyze this 10-K filing excerpt: [insert excerpt here]",
    "Analyze this 10-Q filing excerpt: [insert excerpt here]",
    # Add more prompts as needed
]

jobs = []
for prompt in prompts:
    request = {
        "contents": [types.Content(role="user", parts=[types.Part(text=prompt)])],
        "config": types.GenerateContentConfig(
            response_mime_type="application/json",
            response_schema=response_schema,
            system_instruction=system_prompt,
        ),
    }
    jobs.append(request)

# Create the batch job
batch_job = client.batches.create(
    model=MODEL, src=jobs, config=types.CreateBatchJobConfig(display_name="score-batch")
)

print(f"Batch job created: {jobs}")

# To get results, you can poll the job
# batch_job = client.batches.get(name=batch_job.name)
# When completed, results are in batch_job.dest if specified, or use client.batches.list() to get output

Batch job created: [{'contents': [Content(
  parts=[
    Part(
      text='Analyze this 10-K filing excerpt: [insert excerpt here]'
    ),
  ],
  role='user'
)], 'config': GenerateContentConfig(
  response_mime_type='application/json',
  response_schema=Schema(
    properties={
      'chain_of_thought': Schema(
        type=<Type.STRING: 'STRING'>
      ),
      'confidence': Schema(
        maximum=100.0,
        minimum=0.0,
        type=<Type.INTEGER: 'INTEGER'>
      ),
      'estimated_mw': Schema(
        type=<Type.INTEGER: 'INTEGER'>
      ),
      'score': Schema(
        maximum=10.0,
        minimum=0.0,
        type=<Type.NUMBER: 'NUMBER'>
      )
    },
    required=[
      'chain_of_thought',
      'estimated_mw',
      'score',
      'confidence',
    ],
    type=<Type.OBJECT: 'OBJECT'>
  ),
  system_instruction="""You are a Forensic Financial Analyst specializing in cryptocurrency mining operations and energy infrastructure. Your task is to analyze SEC regulatory filing